In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import io
from google.colab import files
uploaded = files.upload()


Saving data.csv to data.csv


In [2]:
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))
y = df["y"]
df.drop("y", axis = 1, inplace = True)
X = df
print(X)
print(y)
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow import keras
X_train, temp_X_test, y_train, temp_y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_test,X_valid,y_test,y_valid = train_test_split(temp_X_test,temp_y_test,test_size=0.5,random_state=42)


     Unnamed: 0  filename  ...  total_turns  total_distance_travelled
0             0  0000.csv  ...            6                   9090.73
1             1  0001.csv  ...            9                  10718.68
2             2  0002.csv  ...           20                  16852.49
3             3  0003.csv  ...            3                   6664.49
4             4  0004.csv  ...           18                   2050.49
..          ...       ...  ...          ...                       ...
995         995  0995.csv  ...           16                   7526.36
996         996  0996.csv  ...            9                   9753.48
997         997  0997.csv  ...           12                   4342.03
998         998  0998.csv  ...           28                   5372.88
999         999  0999.csv  ...            9                   6130.62

[1000 rows x 19 columns]
0      0
1      0
2      0
3      1
4      0
      ..
995    0
996    1
997    0
998    1
999    1
Name: y, Length: 1000, dtype: int64

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers

In [5]:
tf.random.set_seed(13)
tf.debugging.set_log_device_placement(False)


In [6]:
train_stats = X_train.describe()
train_stats = train_stats.transpose()
def norm(x):
  return (x-train_stats['mean'])/train_stats['std']
X_train = norm(X_train)
X_test =norm(X_test)



In [19]:
def build_model2_three_hidden_layers():
  model = Sequential()
  model.add(Dense(32,input_shape = (X_train.shape[1],)))
  model.add(Dense(32,Activation('relu')))
  model.add(Dense(64,Activation('relu')))
  model.add(Dense(128,Activation('relu')))
  model.add(Dense(1))

  learning_rate= 0.001
  optimizer = optimizers.SGD(learning_rate)
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer = optimizer,metrics=['accuracy'])
  return model
model2 = build_model2_three_hidden_layers()
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                480       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_12 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 12,097
Trainable params: 12,097
Non-trainable params: 0
_________________________________________________________________


In [20]:
X_train.dropna(axis='columns',inplace=True)
X_train
X_valid.dropna(axis='columns',inplace=True)
X_valid
X_test.dropna(axis='columns',inplace=True)
X_test


,Unnamed: 0,feature11,feature12,feature13,feature14,feature4,feature5,feature6,feature7,feature8,feature9,total_distance_travelled,total_stops,total_turns
526,0.063323,-0.050720,-0.129481,-0.050720,0.064184,0.281288,-1.317486,-1.317486,-1.427031,0.019929,0.030750,-0.019410,0.241383,1.793030
557,0.170769,-0.050720,-0.129501,-0.050720,-0.528385,0.373676,0.217256,0.217256,-0.818156,-0.609404,0.030750,0.032917,0.054860,4.150140
290,-0.754648,-0.050714,-0.129487,-0.050714,0.037348,1.287009,-1.046366,-1.046366,-0.222273,0.019929,-0.310919,1.415436,-0.691233,0.573835
141,-1.271079,-0.050720,-0.129450,-0.050720,0.081369,0.480773,-0.504535,-0.504535,-0.553919,0.019929,2.764108,0.762642,-0.285271,-0.645361
239,-0.931413,-0.050705,-0.129501,-0.050705,-0.013682,0.749756,2.207344,2.207344,0.397233,0.019929,-0.310919,0.137966,0.208467,1.142792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,-0.051054,-0.050720,-0.129501,-0.050720,0.606770,1.106913,-1.235621,-1.235621,-1.522856,0.649262,2.764108,0.157397,0.537625,1.224072
554,0.160371,-0.050716,-0.129499,-0.050716,-1.276562,-1.252904,0.622544,0.622544,1.949346,-1.238737,0.030750,-0.592739,-0.109719,0.492555
490,-0.061452,-0.050720,-0.129501,-0.050720,0.621503,-0.792576,-0.603263,-0.603263,0.580381,0.649262,0.372420,-0.445984,-0.866784,0.329996
90,-1.447844,-0.050451,-0.128914,-0.050451,-0.010912,-0.586635,1.069694,1.069694,1.384029,0.019929,-1.335929,1.293733,-0.427906,-0.889200


In [21]:
def build_model1_two_hidden_layers():
  model = Sequential()
  model.add(Dense(32,input_shape = (X_train.shape[1],)))
  #model.add(Dense(32,Activation('relu')))
  #model.add(Dense(64,Activation('relu')))
  #model.add(Dense(128,Activation('relu')))
  model.add(Dense(1))

  learning_rate= 0.0001
  optimizer = optimizers.SGD(learning_rate)
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer = optimizer,metrics=['accuracy'])
  return model
EPOCHS = 500
batch_size = 16
model = build_model1_two_hidden_layers()
model.summary()

with tf.device('/CPU:0'):
  history = model.fit(X_train,y_train,batch_size=batch_size,epochs=EPOCHS,verbose=1,shuffle=False, steps_per_epoch = int(X_train.shape[0]/batch_size),validation_data=(X_valid,y_valid))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 32)                480       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 33        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
37/37 [==============================] - 1s 10ms/step - loss: 0.8289 - accuracy: 0.5602 - val_loss: 0.8093 - val_accuracy: 0.5700
Epoch 2/500
37/37 [==============================] - 0s 4ms/step - loss: 0.8146 - accuracy: 0.5663 - val_loss: 0.8067 - val_accuracy: 0.5750
Epoch 3/500
37/37 [==============================] - 0s 3ms/step - loss: 0.8134 - accuracy: 0.5507 - val_loss: 0.8041 - val_accuracy: 0.5750
Epoch 4/500
37/37 [==============================] - 0s 4ms/step - loss: 0.8199 - accuracy: 0.

In [22]:
loss, accuracy = model.evaluate(X_train,y_train,verbose=1)
print(accuracy)

19/19 [==============================] - 0s 1ms/step - loss: 0.4236 - accuracy: 0.8117
0.8116666674613953


In [23]:
loss, accuracy = model.evaluate(X_valid,y_valid,verbose=1)
print(accuracy)

7/7 [==============================] - 0s 2ms/step - loss: 0.4332 - accuracy: 0.7750
0.7749999761581421


In [24]:
loss, accuracy = model.evaluate(X_test,y_test,verbose=1)
print(accuracy)

7/7 [==============================] - 0s 3ms/step - loss: 0.4249 - accuracy: 0.8300
0.8299999833106995
